In [36]:
#Imports
import pandas as pd
import sklearn as skl
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [37]:
#Read in Data
red_df = pd.read_csv('../Resources/winequality-red.csv', delimiter=';')
white_df = pd.read_csv('../Resources/winequality-white.csv', delimiter=';')

In [38]:
#Splitting
target_red = pd.get_dummies(red_df['quality'])
features_red = red_df.drop(columns='quality')

target_white = pd.get_dummies(white_df['quality'])
features_white = white_df.drop(columns='quality')

X_train_red, X_test_red, y_train_red, y_test_red = train_test_split(features_red, target_red, random_state=42)
X_train_white, X_test_white, y_train_white, y_test_white = train_test_split(features_white, target_white, random_state=42)

In [39]:
#Scaling
X_scaler_red = skl.preprocessing.StandardScaler()
X_scaler_white = skl.preprocessing.StandardScaler()

X_scaler_red.fit(X_train_red)
X_scaler_white.fit(X_train_white)

X_train_scaled_red = X_scaler_red.transform(X_train_red)
X_test_scaled_red = X_scaler_red.transform(X_test_red)

X_train_scaled_white = X_scaler_white.transform(X_train_white)
X_test_scaled_white = X_scaler_white.transform(X_test_white)

In [40]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    model = tf.keras.models.Sequential()
    # Choice of Activation Functions
    activation = hp.Choice('activation', ['relu', 'selu', 'elu', 'softplus', 'softmax', 'tanh', 'softsign'])
    # Choice of first layer Neurons
    model.add(tf.keras.layers.Dense(units=hp.Int('first_unit',
                                                 min_value=1,
                                                 max_value=30,
                                                 step=1), activation=activation, input_dim=len(X_train_red.columns)))
    # Choice of Layers & Layer Neurons
    for i in range(hp.Int('num_layers', 1, 4)):
        model.add(tf.keras.layers.Dense(units=hp.Int('unit_'+str(i),
                                                     min_value=1,
                                                     max_value=20,
                                                     step=1), activation=activation))
    model.add(tf.keras.layers.Dense(len(set(red_df['quality'].values)), activation='softmax'))
    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [41]:
# Import the kerastuner library
import keras_tuner as kt
# Create a `Hyperband()` tuner instance
tuner = kt.Hyperband(
    create_model,
    objective='val_accuracy',
    max_epochs=50,
    hyperband_iterations=2
)

In [42]:
#Find top Models
tuner.search(X_train_scaled_red, y_train_red, epochs=20, validation_data=(X_test_scaled_red, y_test_red))
top_models = tuner.get_best_models(3)

Trial 180 Complete [00h 00m 16s]
val_accuracy: 0.5799999833106995

Best val_accuracy So Far: 0.6200000047683716
Total elapsed time: 00h 25m 09s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [43]:
# Select the top model found
model = top_models[0]

In [44]:
# Evaluate the model using the test data
model_loss, model_accuracy = model.evaluate(X_test_scaled_red, y_test_red, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

13/13 - 1s - loss: 0.8937 - accuracy: 0.6200 - 1s/epoch - 88ms/step
Loss: 0.893694281578064, Accuracy: 0.6200000047683716


In [45]:
model.save('Resources/model.h5')

c:\Users\Bryson\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
